In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset = pd.read_csv('/kaggle/input/bengaluru-house-price-data/Bengaluru_House_Data.csv')
print(dataset.head())

In [ ]:
print(dataset.groupby('area_type')['area_type'].agg('count'))

dataset = dataset.drop(['availability','society','area_type'], axis=1)
print(dataset.head())
print(dataset.shape)

In [ ]:
#Data Cleaning
print(dataset.isnull().sum())

In [ ]:
#add 0 to all null bathroom and balcony values
dataset[['bath','balcony']] = dataset[['bath','balcony']].fillna(0)

In [ ]:
#drop size and location null values
dataset = dataset.dropna()
print(dataset.isnull().sum())
print(dataset.shape)

In [ ]:

#Size column has really weird values
print(dataset['size'].unique())
dataset['size_bhk'] = dataset['size'].apply(lambda x: x.split(' ')[0])  #split by space
dataset = dataset.drop(['size'], axis=1)
print(dataset.head())

In [ ]:
#total_sqft column
print(dataset.total_sqft.unique())  #There are some range values and values in diff format like sq meter, Perch etc
#fetch values which are not integers
print(dataset[dataset['total_sqft'].apply(lambda x: True if len(x) > 7 else False)].head(20))

In [ ]:

#make functions to convert other types into sqft
def convert_dash_value_to_mean(x):
    temp = x.split(' - ')
    if len(temp) == 2:
        return (float(temp[0]) + float(temp[1]))//2
    else:
        return float(temp)

def convert_sq_meter_to_feet(x):
    # 1 sq meter = 10.7639 sq feet
    temp = x.split('Sq. Meter')
    if len(temp) == 2:
        return float(temp[0]) * 10.7639
    else:
        return float(temp[0])

def convert_sq_yards_to_feet(x):
    # 1 sq yard = 9 sq feet
    temp = x.split('Sq. Yards')
    if len(temp) == 2:
        return float(temp[0]) * 9
    else:
        return float(temp[0])

def convert_perch_to_feet(x):
    # 1 perch = 272.25
    temp = x.split('Perch')
    if len(temp) == 2:
        return float(temp[0]) * 272.25
    else:
        return float(temp[0])

def convert_acres_to_feet(x):
    # 1 acres = 4046.86 sq ft
    temp = x.split('Acres')
    if len(temp) == 2:
        return float(temp[0]) * 4046.86
    else:
        return float(temp[0])

def convert_cents_to_feet(x):
    # 1 acres = 435.6 sq ft
    temp = x.split('Cent')
    if len(temp) == 2:
        return float(temp[0]) * 435.6
    else:
        return float(temp[0])

def convert_guntha_to_feet(x):
    # 1 acres = 1089 sq ft
    temp = x.split('Guntha')
    if len(temp) == 2:
        return float(temp[0]) * 1089
    else:
        return float(temp[0])

def convert_ground_to_feet(x):
    # 1 acres = 2400 sq ft
    temp = x.split('Ground')
    if len(temp) == 2:
        return float(temp[0]) * 2400
    else:
        return float(temp[0])

dataset['total_sqft'] = dataset['total_sqft'].apply(lambda x: float(x) if len(x) < 6 else convert_dash_value_to_mean(x) if '-' in x else convert_sq_meter_to_feet(x) if 'Sq. Meter' in x else convert_sq_yards_to_feet(x) if 'Sq. Yards' in x else convert_perch_to_feet(x) if 'Perch' in x else convert_acres_to_feet(x) if 'Acres' in x else convert_cents_to_feet(x) if 'Cent' in x else convert_guntha_to_feet(x) if 'Guntha' in x else convert_ground_to_feet(x) if 'Ground' in x else float(x))

print(dataset.head())


In [ ]:
#converting size_bhk column to float
dataset['size_bhk'] = dataset['size_bhk'].apply(lambda x: float(x))

In [ ]:
#Adding price per sqft to detect outliers
dataset['price_per_sqft'] = dataset['price'].apply(lambda x: float(x*100000))/dataset['total_sqft']    #1 lakh = 100000 INR
print(dataset.head())
print(dataset.price_per_sqft.describe())

In [ ]:
#Adding sqft per bhk to detect outliers

dataset['sqft_per_bhk'] = dataset['total_sqft']/(dataset['size_bhk'] + dataset['bath']+1) 
print(dataset.head())
print(dataset.price_per_sqft.describe())

In [ ]:
min(dataset['sqft_per_bhk'])   #Regular room size is between 100sqft to 1200sqft

In [ ]:
dataset['sqft_per_bhk'] = dataset[~((dataset['sqft_per_bhk']>1200) | (dataset['sqft_per_bhk']<100))]['sqft_per_bhk']
dataset.dropna(inplace=True)
print(dataset.shape)
dataset.head(15)

In [ ]:
max(dataset['price_per_sqft'])

In [ ]:
#Some room sizes based on number of rooms are impossible, so remove them
#per bed-room size < 200 not possible, assuming
dataset = dataset[dataset['total_sqft']//dataset['size_bhk'] > 200]

In [ ]:
#price per sqft min is 23.569 and max is 12000000, which are impossible for generic model, 
q1 = dataset.price_per_sqft.quantile(0.25)
q3 = dataset.price_per_sqft.quantile(0.75)
iqr = q3 - q1
dataset = dataset[(dataset['price_per_sqft']>float(q1-(1.5*iqr))) & (dataset['price_per_sqft']<float(q3+(1.5*iqr)))]
print(dataset.shape)
dataset.head()

## Visualization


In [ ]:
dataset[dataset['size_bhk']==2]['size_bhk'].head()

In [ ]:
dataset['location'].head()

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (15,10)
plt.scatter(dataset[(dataset['size_bhk']==2) & (dataset['location']=='Uttarahalli')]['price'], dataset[(dataset['size_bhk']==2) & (dataset['location']=='Uttarahalli')]['total_sqft'], marker='*', label='2 bhk')
plt.scatter(dataset[(dataset['size_bhk']==3) & (dataset['location']=='Uttarahalli')]['price'], dataset[(dataset['size_bhk']==3) & (dataset['location']=='Uttarahalli')]['total_sqft'], marker='o', label='3 bhk')
plt.xlabel("BHK")
plt.ylabel("Total Price")
plt.legend()
plt.show()

In [ ]:
#Some 2BHK flats have higher price than 3bhk flat, so remove those
def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby(['location']):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('size_bhk'):
            bhk_stats[bhk] = {
                'mean' : np.mean(bhk_df.price_per_sqft),
                'std' : np.std(bhk_df.price_per_sqft),
                'count' : bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('size_bhk'):
            stats = bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)

    return df.drop(exclude_indices, axis='index')

dataset = remove_bhk_outliers(dataset)

print(dataset.shape)

In [ ]:
plt.hist(dataset.price_per_sqft,color='blue',rwidth=0.8)
plt.xlabel("Price Per SqFt")
plt.ylabel("Count")
plt.show()

In [ ]:
#no of bathroom should not be greater than no of bed rooms + 1
#so remove them
print(dataset.groupby('bath')['bath'].agg('count'))

dataset = dataset[(dataset['bath'] <= (dataset['size_bhk'] + 1))]
print(dataset.groupby('bath')['bath'].agg('count'))

In [ ]:
dataset = dataset.drop(['price_per_sqft','sqft_per_bhk'],axis=1)
dataset.head()

In [ ]:
dataset.head()
dataset['location'][0]

In [ ]:
print(dataset.groupby('location')['location'].agg('count'))

In [ ]:
dataset.location = dataset.location.apply(lambda x: x.strip())

In [ ]:
location_stats = dataset.groupby('location')['location'].agg('count').sort_values(ascending=False)


location_stats_less_then_10 = location_stats[location_stats<=10]
dataset.location = dataset.location.apply(lambda x: 'other' if x in location_stats_less_then_10 else x)

In [ ]:
len(dataset.location.unique())

In [ ]:
dummies = pd.get_dummies(dataset.location)
dummies.head(5)

In [ ]:
dummies[dummies['Abbigere']==1]

In [ ]:
n_df = pd.concat([dataset.drop(['location'],axis='columns'),dummies.drop('other',axis='columns')],axis='columns')

In [ ]:

n_df.head()

In [ ]:
n_df[n_df.Abbigere == 1]

In [ ]:
#Model Training
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import GridSearchCV



In [ ]:

X_train, X_test, Y_train, Y_test = train_test_split(n_df.drop(['price'],axis=1),n_df['price'],test_size=0.2) 


In [ ]:
model_params = {
    'linear_regression' : {
        'model': LinearRegression(),
        'params': {
        }
    },

    'lasso' : {
        'model': Lasso(),
        'params': {
            'alpha' : [1,10,20,30,50],
            'max_iter' : [10,15,100],
            'tol' : [0.1]
        }
    },
    
    'ridge' : {
        'model': Ridge(),
        'params': {
            'alpha' : [1,10,20,30,50],
            'max_iter' : [10,15,100]
        }
    },

}

In [ ]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train, Y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_,
    })
    
result_df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
print(result_df)


### Seems like Ridge model is best here

In [ ]:
model = Ridge(alpha=1.0, max_iter=10)
model.fit(X_train, Y_train)

In [ ]:
model.score(X_test,Y_test)

In [ ]:
import joblib
joblib.dump(model,'bengaluru_real_estate_model.ml')

## Importing model and using it

In [ ]:
imp_model = joblib.load('bengaluru_real_estate_model.ml')

def predict_price(location,sqft,bath,bhk):
    loc_index = np.where(n_df.drop(['price'],axis=1).columns==location)[0][0]
    
    x = np.zeros(len(n_df.drop(['price'],axis=1).columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1
        
    return imp_model.predict([x])[0]

predict_price('Vidyaranyapura', 1800, 3, 2)